In [1]:
import ee 
import geemap
import numpy as np
# import eerepr
import folium
import os,sys
from osgeo import gdal
sys.path.append(r'D:\09_Code\Gis_Script')
from functools import partial
from GEE_Func.GEE_Tools import filter_image_collection_by_dates
from GEE_Func.GEE_DataIOTrans import DataIO,Vector_process

geemap.set_proxy(port=10809)
ee.Initialize()
print(geemap.__version__)
print(geemap.__path__)

0.32.1
['c:\\Users\\A\\.conda\\envs\\GEE\\Lib\\site-packages\\geemap']


In [2]:
out_dir = r'D:\S2CloudLess'
# START_DATE = '2020-01-01'          #
# END_DATE = '2020-12-30'            #

date_ranges = [
    ('2021-05-31', '2021-09-30'),
    ('2022-05-31', '2022-09-30'),
    ('2023-05-01', '2023-09-30')
]


Tolerance = 0.9                    # 样本筛选宽容度
Tolerance_cloudPix = 500           # 合成样本空洞容忍量
CLOUD_FILTER = 60                  # 过滤s2 大于指定云量的数据
CLD_PRB_THRESH = 15                # s2cloudless 概率值阈值[0-100],原实验是50
NIR_DRK_THRESH = 0.15              # 非水暗像素判断阈值
CLD_PRJ_DIST = 1                   # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
BUFFER = 50                        # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input
Southest_doom = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/ChinaShp/SouthestRegion')
SETP = Southest_doom.first().geometry()


In [3]:
# # Define the date ranges
# date_ranges = [
#     ('2020-06-01', '2020-06-30'),
#     ('2021-06-01', '2021-06-30'),
#     ('2023-06-01', '2023-06-30')
# ]
# filter_image_collection_by_dates('COPERNICUS/S2_SR_HARMONIZED', date_ranges).filterBounds(SETP)

In [4]:
##--------------------------------------- 云检测以及云阴影检测------------------------------------------------
def get_s2_sr_cld_col1(aoi, date_ranges):
    """筛选S2图像以及S2_cloud图像，并将两个collection连接"""
    # Import and filter S2 SR.
    s2_sr_col = (filter_image_collection_by_dates('COPERNICUS/S2_SR_HARMONIZED', date_ranges)
                 .filterBounds(aoi)
                 .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)).map(partial(clip_AOI,AOI=aoi)))
    # Import and filter s2cloudless.
    s2_cloudless_col = (filter_image_collection_by_dates('COPERNICUS/S2_CLOUD_PROBABILITY', date_ranges)
                        .filterBounds(aoi).map(partial(clip_AOI,AOI=aoi)))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    # 固定用法，将两个collection通过属性值连接起来，s2cloudless整体作为一个属性写入
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def get_s2_sr_cld_col2(aoi, start_date, end_date):
    """筛选S2图像以及S2_cloud图像，并将两个collection连接"""
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)).map(partial(clip_AOI,AOI=aoi)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date).map(partial(clip_AOI,AOI=aoi)))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    # 固定用法，将两个collection通过属性值连接起来，s2cloudless整体作为一个属性写入
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    """Define a function to add the s2cloudless probability layer
    and derived cloud mask as bands to an S2 SR image input."""
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    """Define a function to add dark pixels, 
    cloud projection, and identified shadows as bands to an S2 SR image input.
    Note that the image input needs to be the result of the above add_cloud_bands function
    because it relies on knowing which pixels are considered cloudy ('clouds' band)."""
    # 从 SCL 波段识别水像素, 仅适用于L2A，采用L1C计算MNDWI
    not_water = img.select('SCL').neq(6)

    # 识别非水的暗 NIR 像素(潜在的云阴影像素)。.
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # 确定云投射云影的方向(假设是 UTM 投影)。
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    """Define a function to assemble all of the cloud and cloud shadow components and produce the final mask."""
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()
    # Subset reflectance bands and update their masks, return the result.
    return img.select(['B.*','clouds','dark_pixels','shadows','cloudmask']).updateMask(not_cld_shdw)

def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

def display_cloud_layers(col,AOI):
    # Mosaic the image collection.使用掩码合成集合中的所有图像。普通的mosaic，镶嵌过程中是取最新的影像值
    img = col.mosaic().clip(AOI)

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.centroid求中心点
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
#     m.add_ee_layer(cloud_transform,
#                    {'min': 0, 'max': 1, 'palette': ['white', 'black']},
#                    'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)
    
def display_cloudfree_layers(s2_sr_median,AOI):
    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(s2_sr_median.clip(AOI),
                    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                    'S2 cloud-free mosaic', True, 1, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

def clip_AOI(col,AOI):
    return col.clip(AOI)

def write_dimensions(col,AOI):
    dimensions = ee.Dictionary(ee.List(ee.Dictionary(ee.Algorithms.Describe(col)).get('bands')).get(0)).get('dimensions')
    allNoneZeros = col.select('B1').unmask(0).reduceRegion( **{
                        'reducer': ee.Reducer.allNonZero(),
                        'scale': 10,
                        'maxPixels': 1e12,
                        'bestEffort': True,
                    }).get('B1')
    
    all_NotNone = col.select('B1').unmask(0).reduceRegion(
                        **{
                            'geometry': AOI,
                            'reducer': ee.Reducer.count(),
                            'scale': 10,
                            'maxPixels': 1e12,
                            'bestEffort': True,
                        }).get('B1')
    
    return col.set({'dimensions':ee.List(dimensions).reduce(ee.Reducer.sum()),
                   'all_NotNone':all_NotNone})
    
def cloud_sum(col,AOI):

    cloud_count = col.select('clouds').reduceRegion(
                    **{
                        'geometry': AOI,
                        'reducer': ee.Reducer.sum(),
                        'scale': 10,
                        'maxPixels': 1e12,
                        'bestEffort': True,
                    }
                )
    return col.set({'cloud_count': cloud_count})

def Geemap_export(out_dir,collection=False,image=False,region=None,scale=10):
    if collection:
        # 这里导出时候使用region设置AOI，否则可能因为坐标系问题(未确定)，出现黑边问题
        geemap.ee_export_image_collection(collection,
                                  out_dir=out_dir,
                                  format = "ZIPPED_GEO_TIFF",region=region,scale=scale)
        print('collection save right')
    elif image:
        try:
            filename=os.path.join(out_dir,'Clear'+image.id().getInfo()+'.tif')
        except:
            filename=os.path.join(out_dir,'Cloud.tif')
            
        geemap.ee_export_image(image,
                       filename=filename,
                       scale=scale, region=region, file_per_band=True,timeout=1500)
        print('image save right')
    else:
        print('Erro:collection && image must have one False')

def make_dir(path):
    isExists = os.path.exists(path)
    # 判断结果
    if not isExists:
        os.makedirs(path)
        print(path + ' 创建成功')
    return path

In [5]:
AOI = SETP
s2_sr_cld_col_eval = get_s2_sr_cld_col1(AOI, date_ranges)
# s2_sr_cld_col_eval = get_s2_sr_cld_col2(AOI, START_DATE,END_DATE)
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)#.select(['B.*','clouds','dark_pixels','shadows','cloudmask'])
display_cloud_layers(s2_sr_cld_col_eval_disp,AOI)

# Cloud remove

In [6]:
def export_image_tiles(image, save_path, grid_list,tiles_num, scale):
    tasks = []
    for idx in range(tiles_num):
        tile_path = f"{save_path}_tile_{idx}.tif"
        if os.path.exists(tile_path):
            pass
        else:
            geemap.ee_export_image(image, filename=tile_path, scale=scale, region=grid_list.get(idx), file_per_band=False, timeout=300)
        tasks.append(tile_path)
    return tasks

def merge_tiles(tile_paths, output_path):
    vrt_options = gdal.BuildVRTOptions(resampleAlg='cubic', addAlpha=True)
    vrt = gdal.BuildVRT('/vsimem/temporary.vrt', tile_paths, options=vrt_options)
    gdal.Translate(output_path, vrt)
    vrt = None  # 释放内存

In [7]:
s2_sr_cld_col = get_s2_sr_cld_col1(AOI, date_ranges)
# s2_sr_cld_col = get_s2_sr_cld_col2(AOI, START_DATE,END_DATE)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
s2_sr_median = s2_sr_median.select(['B4','B3','B2']).unmask(0)

In [11]:
Southest_doom_fishnet = geemap.fishnet(SETP, rows=30, cols=45, delta=1)
Num_list = Southest_doom_fishnet.size().getInfo()
Southest_Tibetan_GeoList = Southest_doom_fishnet.toList(Num_list)
SavePath = make_dir(r'D:\S2CloudLess\2021_2023')
os.chdir(SavePath)
for i in range(Num_list):
    region = ee.Feature(Southest_Tibetan_GeoList.get(i)).geometry()
    save_path = f'{i:06d}'+'_'+'S2_cloud_free_mosaic.tif'
    if os.path.exists(save_path):
        print(save_path+' is exist')
    else:
        DataIO.Geemap_export(save_path,
                        s2_sr_median,region=region,rename_image=False)
        try :
            if not os.path.exists(save_path):
                raise FileNotFoundError(f"{save_path} not found after export")
        except Exception as e:
            print(f'直接导出失败: {e}')
            print('开始分块导出...')
            grid_list = Vector_process.split_rectangle_into_grid(region, 3, 3)
            while True:
                tile_paths = export_image_tiles(s2_sr_median, save_path, grid_list, 9, 10)
                # 检查tile——paths全部存在，否则重试
                if np.sum([os.path.exists(each) for each in tile_paths]) == len(tile_paths):
                    break
            merge_tiles(tile_paths, save_path)

000000_S2_cloud_free_mosaic.tif is exist
000001_S2_cloud_free_mosaic.tif is exist
000002_S2_cloud_free_mosaic.tif is exist
000003_S2_cloud_free_mosaic.tif is exist
000004_S2_cloud_free_mosaic.tif is exist
000005_S2_cloud_free_mosaic.tif is exist
000006_S2_cloud_free_mosaic.tif is exist
000007_S2_cloud_free_mosaic.tif is exist
000008_S2_cloud_free_mosaic.tif is exist
000009_S2_cloud_free_mosaic.tif is exist
000010_S2_cloud_free_mosaic.tif is exist
000011_S2_cloud_free_mosaic.tif is exist
000012_S2_cloud_free_mosaic.tif is exist
000013_S2_cloud_free_mosaic.tif is exist
000014_S2_cloud_free_mosaic.tif is exist
000015_S2_cloud_free_mosaic.tif is exist
000016_S2_cloud_free_mosaic.tif is exist
000017_S2_cloud_free_mosaic.tif is exist
000018_S2_cloud_free_mosaic.tif is exist
000019_S2_cloud_free_mosaic.tif is exist
000020_S2_cloud_free_mosaic.tif is exist
000021_S2_cloud_free_mosaic.tif is exist
000022_S2_cloud_free_mosaic.tif is exist
000023_S2_cloud_free_mosaic.tif is exist
000024_S2_cloud_

In [9]:
ee.Feature(Southest_Tibetan_GeoList.get(i)).geometry()

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Feature.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "List.get",
          "arguments": {
            "index": {
              "constantValue": 807
            },
            "list": {
              "functionInvocationValue": {
                "functionName": "Collection.toList",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
                            "functionName": "Collection.flatten",
                            "arguments": {
                              "collection": {
                                "functionInvocationValue": {
                                  "functionName": "Collection",
                                  "arguments": {
                                    "features": {
                                      "functionInvocationValue": {
                                        "functionName": "List.map",
                                        "arguments": {
                                          "baseAlgorithm": {
                                            "functionDefinitionValue": {
                                              "argumentNames": [
                                                "_MAPPING_VAR_1_0"
                                              ],
                                              "body": {
                                                "functionInvocationValue": {
                                                  "functionName": "Collection",
                                                  "arguments": {
                                                    "features": {
                                                      "functionInvocationValue": {
                                                        "functionName": "List.map",
                                                        "arguments": {
                                                          "baseAlgorithm": {
                                                            "functionDefinitionValue": {
                                                              "argumentNames": [
                                                                "_MAPPING_VAR_0_0"
                                                              ],
                                                              "body": {
                                                                "functionInvocationValue": {
                                                                  "functionName": "Element.set",
                                                                  "arguments": {
                                                                    "key": {
                                                                      "constantValue": "east"
                                                                    },
                                                                    "object": {
                                                                      "functionInvocationValue": {
                                                                        "functionName": "Element.set",
                                                                        "arguments": {
                                                                          "key": {
                                                                            "constantValue": "north"
                                                                          },
                                                                          "object": {
                                                                            "functionInvocationValue": {
              

In [10]:
# Create a folium map object.
Map = geemap.Map()
Map.centerObject(AOI.centroid(10), zoom=15)

# Add layers to the folium map.
Map.add_ee_layer(s2_sr_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                'S2 cloud-free mosaic')


# Display the map.
Map

Map(center=[29.790744928311963, 95.21823162973041], controls=(WidgetControl(options=['position', 'transparent_…